In [ ]:
!pip install -q pretrainedmodels

In [ ]:
import pretrainedmodels

def model_f(pretrained=True,**kwargs):
    return pretrainedmodels.se_resnext50_32x4d(num_classes=1000,pretrained='imagenet')

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from fastai.vision.all import *
from PIL import Image
from sklearn.metrics import roc_auc_score

import os
!ls /kaggle/input/plant-pathology-2020-fgvc7

In [ ]:
path = Path('/kaggle/input/plant-pathology-2020-fgvc7')
train = pd.read_csv(path/'train.csv')
test = pd.read_csv(path/'test.csv')

# inverse operation of get_dummies
train['label'] = train.set_index('image_id').idxmax(axis=1).reset_index()[0]
train

# pseudo labelling

In [ ]:
pseudos = ['Test_1149', 'Test_1158','Test_1321','Test_1590','Test_1634','Test_168','Test_1728','Test_1801','Test_211','Test_282','Test_506','Test_543','Test_714','Test_727','Test_921','Test_932','Test_987','Test_991']


In [ ]:
# pseudos = ['Test_1158', 'Test_714', 'Test_1634', 'Test_921', 'Test_168', 'Test_211', 'Test_1801', 'Test_1149', 'Test_987']
c=['image_id', 'healthy', 'multiple_diseases', 'rust', 'scab', 'label']
pseudos = pd.DataFrame([(x, 0,1,0,0,'multiple_diseases') for x in pseudos], columns=c)
pseudos

In [ ]:
train = pd.concat([train, pseudos], axis=0)

In [ ]:
len(train)

# dealing with duplicates
* Test Test -> do nothing
* Train Test -> use training label as prediction
* Train Train -> remove duplicates

In [ ]:
dups = [["Test_1800", "Train_558"], ["Train_616", "Train_966"], ["Test_383", "Test_635"], ["Train_500", "Train_647"], ["Test_1691", "Test_683"], ["Test_431", "Test_776"], ["Test_218", "Train_1598"], ["Test_611", "Train_1698"], ["Train_909", "Train_914"], ["Test_1688", "Train_1154"], ["Test_700", "Train_1142"], ["Test_1047", "Train_1733"], ["Train_592", "Train_782"], ["Train_1675", "Train_417"], ["Test_1450", "Train_761"], ["Test_989", "Train_1347"], ["Test_1117", "Train_105"], ["Test_119", "Train_772"], ["Test_1503", "Test_232"], ["Test_1264", "Test_708"], ["Test_483", "Train_1407"], ["Test_1212", "Test_570"], ["Train_174", "Train_1817"], ["Test_1722", "Test_889"], ["Test_693", "Train_1277"], ["Test_1494", "Train_937"], ["Test_938", "Train_861"], ["Test_53", "Train_114"], ["Test_581", "Train_1615"], ["Train_1731", "Train_591"], ["Test_1802", "Train_574"], ["Train_665", "Train_694"], ["Train_1634", "Train_921"], ["Train_1115", "Train_1683"], ["Train_1661", "Train_815"], ["Test_1062", "Train_1368"], ["Test_1720", "Test_459"], ["Test_1561", "Test_443"], ["Train_759", "Train_828"], ["Test_1671", "Test_899"], ["Train_1581", "Train_1613"], ["Test_768", "Train_851"], ["Test_1203", "Test_1600"], ["Test_797", "Train_1585"], ["Test_1163", "Test_542"], ["Test_886", "Train_1541"], ["Test_1011", "Train_608"], ["Train_1173", "Train_379"], ["Test_1588", "Test_376"], ["Test_1675", "Train_1066"], ["Test_1020", "Train_683"], ["Test_150", "Test_1501"], ["Train_1678", "Train_931"], ["Test_965", "Train_1341"], ["Train_1587", "Train_1593"], ["Train_341", "Train_763"], ["Test_895", "Test_979"], ["Train_162", "Train_356"], ["Test_1407", "Train_1703"], ["Test_1322", "Train_1600"], ["Test_1524", "Train_919"], ["Train_1150", "Train_1365"], ["Train_1392", "Train_438"], ["Test_829", "Train_1505"], ["Test_826", "Train_847"], ["Train_115", "Train_1709"]]
dups = pd.DataFrame(dups, columns=['col1', 'col2'])
dups.head()
dups['col1t'] = dups['col1'].str.split('_').str.get(0)
dups['col2t'] = dups['col2'].str.split('_').str.get(0)
dups.head()

In [ ]:
# remove train_dups['col2'] images from training
train_dups = dups[(dups.col1t == 'Train') & (dups.col1t == dups.col2t)].merge(train, left_on='col1', right_on='image_id').merge(train, left_on='col2', right_on='image_id')
train_dups.head()

In [ ]:
confused = train_dups[train_dups.label_x != train_dups.label_y]
confused

In [ ]:
print(f'list of duplicates train / train: {len(train_dups)}')
print(len(train))
train = train[~train.image_id.isin(list(train_dups['col2']))].reset_index(drop=True)
print(len(train))
train = train[~train.image_id.isin(list(confused['col1']))].reset_index(drop=True)
train = train[~train.image_id.isin(list(confused['col2']))].reset_index(drop=True)
print(len(train))

In [ ]:
# do nothing for these
dups[(dups.col1t == 'Test') & (dups.col1t == dups.col2t)]

In [ ]:
# use these to put 1 in the prediction at the end
solutions = dups[dups.col1t != dups.col2t].merge(train, left_on='col2', right_on='image_id').sort_values(by='label')
solutions.head()

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(shuffle=True, random_state=42)

for i, (train_index, val_index) in enumerate(skf.split(train, train['label'])):
    print(train_index.shape, val_index.shape)
    train.loc[val_index, 'fold'] = i

In [ ]:
# oversampling
train = train.iloc[list(train.index) + [x for x in train[train.label == 'multiple_diseases'].index for j in range(0,2)]]
train = train.reset_index(drop=True)

In [ ]:
folds = range(0,3)
# folds = [2]

In [ ]:
R1=(400,600) # done on CPU to fit all on the GPU
R2=(320,512) # done on GPU on batch
# R1=450
# R2=380
# 380 for effnet b4

TTA=True
# TTA=False
EPOCH1=15
# EPOCH1=1
# EPOCH2=7
EPOCH2=-1

LR1=5e-2
LR2=slice(1e-7, 1e-4)

BS=16

##########

tfms = [Rotate(max_deg=30), 
        Zoom(),      
        Brightness(), 
        Flip(), 
        Contrast(), 
#         RandomCrop(R2),
        Resize(R2), 
        Normalize.from_stats(*imagenet_stats)]

comp = setup_aug_tfms(tfms)

for fold in folds:

    plant = DataBlock(blocks=(ImageBlock, CategoryBlock),
                      splitter=IndexSplitter(train[train['fold'] == fold].index),
                      get_x=ColReader('image_id', pref=path/"images", suff='.jpg'),
                      get_y=ColReader('label'),
                      item_tfms=Resize(R1),
                      batch_tfms=comp)

    dls = plant.dataloaders(train, bs=BS, seed=42)
    
    learn = cnn_learner(dls, 
                        model_f, 
                        pretrained=True, 
                        loss_func=LabelSmoothingCrossEntropy(), 
                        cbs=SaveModelCallback(monitor='roc_auc_score', comp=np.greater),
                        model_dir='/kaggle/working',
                        metrics=[RocAuc(), error_rate])
    
    print(f'-------- {fold} ---------')
    if EPOCH1 > 0:
        learn.fine_tune(EPOCH1, 
                        base_lr=LR1,
                        cbs=SaveModelCallback(monitor='roc_auc_score', comp=np.greater))
        learn.recorder.plot_loss()
        
    if EPOCH2 > 0:
        learn.unfreeze()    
        learn.fit_one_cycle(EPOCH2, 
                            lr_max=LR2,
                            cbs=SaveModelCallback(monitor='roc_auc_score', comp=np.greater))
        learn.recorder.plot_loss()
    
    learn.save(f'seresnext50-fold-{fold}')
    
    tst_dl = dls.test_dl(test)
    
    if TTA:
        preds, _ = learn.tta(dl=tst_dl)
    else:
        preds, _ = learn.get_preds(dl=tst_dl)
    
    cols=['image_id'] + list(dls.vocab)
    res = pd.concat([test, pd.DataFrame(preds,columns = learn.dls.vocab)],axis=1)[cols]
    res.to_csv(f'submission_resnext_{fold}.csv',index=False)
    
    print('confident multiple_diseases')
    my_ids = res[res['multiple_diseases'] > 0.9].sort_values(by='multiple_diseases').image_id.values
    print(my_ids)
    
    res = learn.get_preds()
    cols= dls.vocab
    r = pd.DataFrame(res[0], columns=cols)
    
    valid_with_preds = pd.concat([train[train['fold'] == fold].reset_index(drop=True), r], axis=1)
    
    print(roc_auc_score(train[train['fold'] == fold][cols], r))
    
    for col in cols:
        score = roc_auc_score(train[train['fold'] == fold][col], r[col])
        print(f'{col} -> {score}')
        
    valid_with_preds.to_csv(f'validation_resnext_{fold}.csv',index=False)

In [ ]:
from scipy.stats import rankdata

if len(folds) > 1:

    my_df = {}
    for fold in folds:
        my_df[fold] = pd.read_csv(f'submission_resnext_{fold}.csv').set_index('image_id')

    # for each target, for each fold, compute the rank of the solution
    for x in ['healthy','multiple_diseases','rust','scab']:
        for fold in folds:
            my_df[fold][x] = rankdata(my_df[fold][x], method='min')

    # sum the ranks in the 1st df and average (not necessary, just for coherence)
    for x in ['healthy','multiple_diseases','rust','scab']:
        for fold in folds[1:]:
            my_df[folds[0]][x] += my_df[fold][x]
        my_df[folds[0]][x] = my_df[folds[0]][x] / 5

    my_df[folds[0]].reset_index().to_csv('submission_resnext.csv', index=False)

else:
    
    print('no need for rankdata')